In [2]:
import numpy as np
import matplotlib.pyplot as plt
import netCDF4
from matplotlib import colors,ticker
import matplotlib.gridspec as gridspec
# from path import Path
from scipy.ndimage import uniform_filter1d
#make path to the module active
import sys
sys.path.append('/glade/u/home/blazg/scripts/modules_python/')

from make_cmap import *
from press_to_alt import * #for pressure to altitude transofrmation
import sam #for module ncopen
import util


cldmap = make_colormap('CBR_wet.rgb')
icemap = make_colormap('ice_new.rgb')

path = '/glade/campaign/univ/uwas0108/blazg/obs_data/cirrus-guideII-database/'
timeind= slice(None)
levind = slice(None)
lonind = slice(None)
latind = slice(None)

varnames = ['time','y','x','z']



In [3]:
import fnmatch
import os

path = '/glade/campaign/univ/uwas0108/blazg/obs_data/cirrus-guideII-database/'
campaigns = ['*attrex*','*posidon*','*CONTRAST*']
allp = []

for filename in os.listdir(path):
    for campaign in campaigns:
        if fnmatch.fnmatch(filename, campaign):
            #allp.append(filename) only the names
            allp.append(os.path.join(path, filename))


HERE NOW ICNC VS REFFI

In [8]:
insitu=0
liqor=0

In [15]:
varloop = ['rei']#,'omega','convf']
ybins = 10**np.linspace(-5,2,71)

hist_var = {}
hist_freq = {}
hist_norm1 = {}

varnames = ['BEST:PRESS','BEST:TEMP','BEST:N_ice','BEST:IWC','BEST:R_ice'] 

hist_counts={}


# allcounts = np.zeros((3,len(xbins)-1,len(ybins)-1))



for d in range(len(allp)):
    #print(d,'campaign #')
    # hist_counts = np.zeros((3,len(xbins)-1,len(ybins)-1))
    
    ncfile = netCDF4.Dataset(allp[d])
    data_3d = {}

    #load variables
    for var in varnames:
        #???data_3d[var] = np.array(ncfile.variables[var][5040:])
        data_3d[var] = np.array(ncfile.variables[var][:])

    
    # pres=data_3d['BEST:PRESS'].squeeze() #hPa
    # iwc_ppmv = data_3d['BEST:IWC']
    # iwc=data_3d['BEST:IWC']*18.015/22.41 * 1e-3 * pres/1013  #from ppmv to g/m3
    # print(iwc.shape)
    icnc=data_3d['BEST:N_ice'].squeeze()  #N/cm-3
    rei=data_3d['BEST:R_ice'].squeeze() #microm
    temp0=data_3d['BEST:TEMP'].squeeze() #K

    #running mean here
    rei= uniform_filter1d( rei, size=18)
    icnc= uniform_filter1d( icnc, size=18)
    
    #iwc[temp0<0]  = -9999
    #iwc[temp0>283]= -9999
    # iwc[temp0>233.15]= -9999
    rei[temp0>233.15]= -9999
    icnc[temp0>233.15]= -9999


    if insitu==1:
        print('now only what <0.1 PPMV ICE')
        # iwc[iwc_ppmv>0.1] = -9999
        icnc[iwc_ppmv>0.1] = -9999
        # rei[iwc_ppmv>0.1] = -9999
    if liqor==1:
        print('now only what >10 PPMV ICE')
        # iwc[iwc_ppmv<10.] = -9999
        icnc[iwc_ppmv<10.] = -9999
        # rei[iwc_ppmv<10.] = -9999

    for c,car in enumerate(varloop):

        plotvar_x = rei.ravel()
        plotvar_y = icnc.ravel()

        # if c==0:
        #     xbins = np.arange(0,141,2)
        #     xplot4=(xbins[:-1]+xbins[1:])/2.
            
        # hist_counts[c] = np.histogram2d(plotvar_x, plotvar_y, bins=[xbins,ybins], normed=False)[0] 
    # allcounts = allcounts +    hist_counts
    # print(np.sum(allcounts[0]),'after')

write=1
######################################################################################
######################SAVE VARIABLES in an NC FILE###################################
# yplot =(ybins[:-1]+ybins[1:])/2.
      
if (write == 1):
    
    ##############WRITE IN A NC FILE##################################################

    ############################################################################
    from netCDF4 import Dataset as NetCDFFile
    
    savepath = '/glade/work/sturbeville/obs/ATTREX_POSIDON_CONTRAST/processed/'
    print('saving')
    ncfile = NetCDFFile(savepath+'ATT_POS_CONT_microguideII_icnc_rei_runningmean18x.nc', 'w',  format='NETCDF4_CLASSIC') 

    # icnc = ncfile.createDimension('icnc', len(yplot) )
    icnc = ncfile.createDimension('icnc', len(plotvar_y) )
    rad = ncfile.createDimension('rad', len(plotvar_x) )

    # Create coordinate variables for 2-dimensions
    rad = ncfile.createVariable('ice mass radius', np.float32, ('rad',)) 
    icnc = ncfile.createVariable('ice number concentration', np.float32, ('icnc',)) 


    # Create the actual 2-d variable
    # rei_bin    = ncfile.createVariable('binned in-cloud ice crystal radius', np.dtype('float32').char, ('icnc','rad'))

    # Variable Attributes  
    # rei_bin.units      = 'micron'  
   
    #assign values
    icnc[:] = plotvar_y
    rad[:]  = plotvar_x
#     icnc[:]           = yplot
#     rad[:]            = xplot4
    
#     rei_bin[:]       = np.flipud(allcounts[0][:,::-1].T)

    ncfile.close()    
print("saved")

saving
saved


In [28]:
np.sum(allcounts[0])

251528.0

In [30]:
allcounts.shape


(3, 70, 70)

In [34]:
yplot

array([1.12946271e-05, 1.42190930e-05, 1.79007775e-05, 2.25357437e-05,
       2.83708205e-05, 3.57167468e-05, 4.49647202e-05, 5.66072289e-05,
       7.12642790e-05, 8.97164117e-05, 1.12946271e-04, 1.42190930e-04,
       1.79007775e-04, 2.25357437e-04, 2.83708205e-04, 3.57167468e-04,
       4.49647202e-04, 5.66072289e-04, 7.12642790e-04, 8.97164117e-04,
       1.12946271e-03, 1.42190930e-03, 1.79007775e-03, 2.25357437e-03,
       2.83708205e-03, 3.57167468e-03, 4.49647202e-03, 5.66072289e-03,
       7.12642790e-03, 8.97164117e-03, 1.12946271e-02, 1.42190930e-02,
       1.79007775e-02, 2.25357437e-02, 2.83708205e-02, 3.57167468e-02,
       4.49647202e-02, 5.66072289e-02, 7.12642790e-02, 8.97164117e-02,
       1.12946271e-01, 1.42190930e-01, 1.79007775e-01, 2.25357437e-01,
       2.83708205e-01, 3.57167468e-01, 4.49647202e-01, 5.66072289e-01,
       7.12642790e-01, 8.97164117e-01, 1.12946271e+00, 1.42190930e+00,
       1.79007775e+00, 2.25357437e+00, 2.83708205e+00, 3.57167468e+00,
      